<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_ATH_TTM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-11-14 15:50:14


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

mypf = mypf[mypf['LatestQtr'] == 0]
#mypf = mypf[mypf['v80'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','PGHH','SIEMENS', 'GILLETTE', 'NESTLEIND', 'SANOFI', 'ADANITRANS', 'AMARAJABAT']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100[0:50]
#stock_ids = ['DIVISLAB', 'PREMIERPOL']
print('Stocks to be scanned:',len(stock_ids))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Stocks to be scanned: 50


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_latest_analysis = pd.DataFrame()

for stock_name in stock_ids:

  print(stock_name)

  link_blueprint = "https://www.screener.in/company/"

  stock_list = ['3MINDIA', 'AUBANK', 'COLPAL', 'GLAND', 'LUXIND', 'WHIRLPOOL' , 'IEX', 'ROSSARI', 'VALIANTORG', 'VINATIORGA', 'TATAELXSI',
                'AMARAJABAT', 'EQUITAS', 'AAVAS']

  if stock_name in stock_list:
      actual_link = link_blueprint + stock_name.upper()
  else:
      actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"

  def get_list(actual_link):
      page = requests.get(actual_link)
      link_soup = BeautifulSoup(page.content, 'html.parser')

      test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')

      while("" in test_list) :
          test_list.remove("")
      while("\r" in test_list) :
          test_list.remove("\r")
      while(" \r" in test_list) :
          test_list.remove(" \r")

      return test_list
  #print(actual_link)
  test_list = get_list(actual_link)

  def get_yoy(test_list):
    #print(len(test_list))
    n = len(test_list)

    if n < 100:
        actual_link = link_blueprint + stock_name.upper()
        test_list = get_list(actual_link)
        #print(len(test_list))

    #print(test_list)
    if 'TTM' not in test_list:
        end_year = 'Mar 2023'
    else:
        end_year = 'TTM'

    df_yoy = pd.DataFrame()

    #print(test_list.index('Sep 2023'))
    s = test_list.index(end_year) +1 - test_list.index('Mar 2017')

    df_yoy['Year'] = test_list[test_list.index('Mar 2017'):test_list.index(end_year)+1]
    df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
    df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

    df_yoy.set_index('Year', inplace=True)
    df_yoy = df_yoy.T
    df_yoy['Stock'] = stock_name.upper() + '.NS'
    #df_yoy
    return df_yoy
  page = requests.get(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')
  scan_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')
  #print(scan_list.index('Sep 2023'))
  try:
    scan_list.index('Sep 2023') > 30
    df_latest_analysis = pd.concat([df_latest_analysis, get_yoy(test_list)])
  except ValueError:
      print(stock_name + ' = FAILED =')
      pass  # do nothing!

ABBOTINDIA
ABBOTINDIA = FAILED =
ALKYLAMINE
ALKYLAMINE = FAILED =
APEX
APEX = FAILED =
ASTRAZEN
ASTRAZEN = FAILED =
BANDHANBNK
BANDHANBNK = FAILED =
BAYERCROP
BAYERCROP = FAILED =
BIRLAMONEY
BIRLAMONEY = FAILED =
CAMPUS
CAMPUS = FAILED =
CAMS
CAMS = FAILED =
CLEAN
CLEAN = FAILED =
COALINDIA
CROMPTON
CROMPTON = FAILED =
CUB
CUB = FAILED =
DREAMFOLKS
DREAMFOLKS = FAILED =
DRREDDY
EICHERMOT
EMAMILTD
EQUITAS
EQUITAS = FAILED =
EQUITASBNK
EQUITASBNK = FAILED =
ERIS
FEDERALBNK
FINCABLES
FINEORG
GAIL
GICRE
GLAND
GLAXO
GODREJCP
GODREJIND
GRASIM
HATHWAY
HAVELLS
HCLTECH
HDFCAMC
HDFCBANK
HDFCLIFE
HEROMOTOCO
HINDALCO
HINDPETRO
HINDUNILVR
HONAUT
HONAUT = FAILED =
IBREALEST
IBULHSGFIN
IBULHSGFIN = FAILED =
ICICIBANK
ICICIGI
ICICIGI = FAILED =
ICICIPRULI
IEX
IGL
INDIANB
INDIGO


In [5]:
print('Stocks updated:', df_latest_analysis.shape[0]/2)
if df_latest_analysis.shape[0] > 0:
  lastestq_stocks = list(df_latest_analysis['Stock'].unique())
  len(lastestq_stocks)

Stocks updated: 32.0


In [6]:
df_latest_analysis

Year,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,TTM,Stock
Sales,"78,164","85,244","99,586","96,080","90,026","109,715","138,252","142,081",COALINDIA.NS
Profit,"14,446","10,770","27,127","24,071","18,009","23,616","38,001","37,798",COALINDIA.NS
Sales,"14,196","14,281","15,448","17,517","19,048","21,545","24,670","26,766",DRREDDY.NS
Profit,"1,554","1,350","2,336","1,886","2,884","3,061","6,048","6,734",DRREDDY.NS
Sales,"7,033","8,965","9,797","9,154","8,720","10,298","14,442","15,626",EICHERMOT.NS
...,...,...,...,...,...,...,...,...,...
Profit,904,"1,096","1,269","1,556","1,422","1,953","2,122","2,322",IGL.NS
Sales,"10,891","10,851","12,167","13,798","23,439","22,129","24,717","28,420",INDIANB.NS
Profit,"1,060","1,445",359,"1,377","3,115","3,253","5,963","8,235",INDIANB.NS
Sales,"18,580","23,021","28,497","35,756","14,641","25,931","54,446","60,720",INDIGO.NS


In [7]:
if df_latest_analysis.shape[0] > 0:
  #df_analysis.columns
  df_latest_analysis = df_latest_analysis.reset_index()
  df_latest_analysis.rename({'TTM': 'Latest', 'index': 'Type'}, axis=1, inplace=True)
  cols = [ 'Mar 2017','Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Mar 2023', 'Latest']
  #df_analysis[cols] = df_analysis[cols].apply(pd.to_numeric)
  for col in cols:
      df_latest_analysis[col] = df_latest_analysis[col].str.replace(',', '').astype(float)

In [8]:
df_latest_analysis

Year,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Latest,Stock
0,Sales,78164.0,85244.0,99586.0,96080.0,90026.0,109715.0,138252.0,142081.0,COALINDIA.NS
1,Profit,14446.0,10770.0,27127.0,24071.0,18009.0,23616.0,38001.0,37798.0,COALINDIA.NS
2,Sales,14196.0,14281.0,15448.0,17517.0,19048.0,21545.0,24670.0,26766.0,DRREDDY.NS
3,Profit,1554.0,1350.0,2336.0,1886.0,2884.0,3061.0,6048.0,6734.0,DRREDDY.NS
4,Sales,7033.0,8965.0,9797.0,9154.0,8720.0,10298.0,14442.0,15626.0,EICHERMOT.NS
...,...,...,...,...,...,...,...,...,...,...
59,Profit,904.0,1096.0,1269.0,1556.0,1422.0,1953.0,2122.0,2322.0,IGL.NS
60,Sales,10891.0,10851.0,12167.0,13798.0,23439.0,22129.0,24717.0,28420.0,INDIANB.NS
61,Profit,1060.0,1445.0,359.0,1377.0,3115.0,3253.0,5963.0,8235.0,INDIANB.NS
62,Sales,18580.0,23021.0,28497.0,35756.0,14641.0,25931.0,54446.0,60720.0,INDIGO.NS


In [9]:
if df_latest_analysis.shape[0] > 0:
  df_latest_analysis['Max'] = df_latest_analysis.max(axis=1)
  df_latest_analysis['Diff%'] = round((df_latest_analysis['Max'] - df_latest_analysis['Latest'])*100/df_latest_analysis['Max'],2)

<ipython-input-9-ff4f08accfa3>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_latest_analysis['Max'] = df_latest_analysis.max(axis=1)


In [10]:
df_latest_analysis

Year,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Latest,Stock,Max,Diff%
0,Sales,78164.0,85244.0,99586.0,96080.0,90026.0,109715.0,138252.0,142081.0,COALINDIA.NS,142081.0,0.00
1,Profit,14446.0,10770.0,27127.0,24071.0,18009.0,23616.0,38001.0,37798.0,COALINDIA.NS,38001.0,0.53
2,Sales,14196.0,14281.0,15448.0,17517.0,19048.0,21545.0,24670.0,26766.0,DRREDDY.NS,26766.0,0.00
3,Profit,1554.0,1350.0,2336.0,1886.0,2884.0,3061.0,6048.0,6734.0,DRREDDY.NS,6734.0,0.00
4,Sales,7033.0,8965.0,9797.0,9154.0,8720.0,10298.0,14442.0,15626.0,EICHERMOT.NS,15626.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
59,Profit,904.0,1096.0,1269.0,1556.0,1422.0,1953.0,2122.0,2322.0,IGL.NS,2322.0,0.00
60,Sales,10891.0,10851.0,12167.0,13798.0,23439.0,22129.0,24717.0,28420.0,INDIANB.NS,28420.0,0.00
61,Profit,1060.0,1445.0,359.0,1377.0,3115.0,3253.0,5963.0,8235.0,INDIANB.NS,8235.0,0.00
62,Sales,18580.0,23021.0,28497.0,35756.0,14641.0,25931.0,54446.0,60720.0,INDIGO.NS,60720.0,0.00


In [11]:
if df_latest_analysis.shape[0] > 0:
  df_master_analysis = pd.read_csv('/content/drive/My Drive/data/stocks/myProspects-ATH-TTM.csv')
  df_master_analysis.columns = df_latest_analysis.columns
  df_master_analysis = df_master_analysis[~df_master_analysis['Stock'].isin(lastestq_stocks)]
  df_master_analysis = pd.concat([df_master_analysis, df_latest_analysis])
  df_master_analysis.to_csv('/content/drive/My Drive/data/stocks/myProspects-ATH-TTM.csv', index=False)

In [12]:
if df_latest_analysis.shape[0] > 0:
  df_tmp = df_latest_analysis[df_latest_analysis['Diff%'] <= 5]
  tmp = pd.DataFrame(df_tmp['Stock'].value_counts())
  len(tmp[tmp['Stock']>1].index)
  star_stocks = list(tmp[tmp['Stock']>1].index)
  print(len(star_stocks))

20


In [13]:
mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')
mypf.loc[mypf['Stock'].isin(lastestq_stocks), 'LatestQtr'] = 1
mypf.loc[mypf['Stock'].isin(lastestq_stocks), 'StarStock'] = 0
mypf.loc[mypf['Stock'].isin(star_stocks), 'StarStock'] = 1
mypf.to_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv', index=False)